In [2]:
%config Completer.use_jedi = False

In [3]:
import textworld

In [26]:
from typing import Mapping, Any

import numpy as np

import textworld.gym


class RandomAgent(textworld.gym.Agent):
    """ Agent that randomly selects a command from the admissible ones. """
    def __init__(self, seed=1234):
        self.seed = seed
        self.rng = np.random.RandomState(self.seed)

    @property
    def infos_to_request(self) -> textworld.EnvInfos:
        return textworld.EnvInfos(admissible_commands=True)
    
    def act(self, obs: str, score: int, done: bool, infos: Mapping[str, Any]) -> str:
        return self.rng.choice(infos["admissible_commands"])


In [53]:
import numpy as np

#from TextWorld.benchmark.benchmark import *
#from TextWorld.benchmark.agent_template import *

/bin/sh: 1: tw-benchmark: not found


In [51]:
def evaluate(agent, game):
    env = textworld.start(game)
    log.debug("Using {}".format(env.__class__.__name__))
    agent.reset(env)

    start_time = time.time()
    game_state = env.reset()
    log.debug("Environment reset.\n{}\n".format(env.render(mode="text")))

    max_score = game_state.max_score
    nb_losts = 0
    highscore = 0
    score = 0
    done = False

    for step in range(1, args.nb_steps + 1):
        action = agent.act(game_state, score, done)
        game_state, score, done = env.step(action)

        msg = "{:5d}. Time: {:9.2f}\tScore: {:3d}\tMove: {:5d}\tAction: {:20s}"
        msg = msg.format(step, time.time() - start_time, game_state.score, game_state.moves, action)
        log.info(msg)
        log.debug(env.render(mode="text"))

        if done:
            highscore = max(score, highscore)

            if game_state.won:
                if highscore == max_score:
                    break  # No reason to play that game more.
            elif game_state.lost:
                nb_losts += 1
            else:
                assert True, "Games should either end with a win or a fail."

            # Replay the game in the hope of achieving a better score.
            game_state = env.reset()
            log.debug("Environment reset.\n{}\n".format(env.render(mode="text")))

    env.close()

    # Keep highest score.
    highscore = max(score, highscore)

    return step, nb_losts, highscore, max_score, time.time() - start_time

usage: ipykernel_launcher.py [-h] [--games game [game ...]] [--agent AGENT] [--nb-steps NB_STEPS]
                             [--summary_out_file SUMMARY_OUT_FILE] [--log_file LOG_FILE] [-v] [-vv]
ipykernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-991559dd-9284-461d-ab0e-5148f2d296fa.json


SystemExit: 2

/usr/local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3339: UserWarning:

To exit: use 'exit', 'quit', or Ctrl-D.



In [52]:
evaluate(RandomAgent(), "./textplayer/games/acorncourt.z5". args)

AttributeError: 'str' object has no attribute 'args'

In [28]:
import os
from glob import glob

import gym
import textworld.gym


def play(agent, path, max_step=100, nb_episodes=10, verbose=True):
    infos_to_request = agent.infos_to_request
    infos_to_request.max_score = True  # Needed to normalize the scores.
    
    gamefiles = [path]
    if os.path.isdir(path):
        gamefiles = glob(os.path.join(path, "*.ulx"))
        
    env_id = textworld.gym.register_games(gamefiles,
                                          request_infos=infos_to_request,
                                          max_episode_steps=max_step)
    env = gym.make(env_id)  # Create a Gym environment to play the text game.
    if verbose:
        if os.path.isdir(path):
            print(os.path.dirname(path), end="")
        else:
            print(os.path.basename(path), end="")
        
    # Collect some statistics: nb_steps, final reward.
    avg_moves, avg_scores, avg_norm_scores = [], [], []
    for no_episode in range(nb_episodes):
        obs, infos = env.reset()  # Start new episode.

        score = 0
        done = False
        nb_moves = 0
        while not done:
            command = agent.act(obs, score, done, infos)
            obs, score, done, infos = env.step(command)
            nb_moves += 1
        
        agent.act(obs, score, done, infos)  # Let the agent know the game is done.
                
        if verbose:
            print(".", end="")
        avg_moves.append(nb_moves)
        avg_scores.append(score)
        avg_norm_scores.append(score / infos["max_score"])

    env.close()
    msg = "  \tavg. steps: {:5.1f}; avg. score: {:4.1f} / {}."
    if verbose:
        if os.path.isdir(path):
            print(msg.format(np.mean(avg_moves), np.mean(avg_norm_scores), 1))
        else:
            print(msg.format(np.mean(avg_moves), np.mean(avg_scores), infos["max_score"]))
    

In [27]:
textworld.play("./textplayer/games/acorncourt.z5")


GREAT.  THEY'VE DONE IT TO ME AGAIN.  You think to yourself. THEY'VE STUCK ME IN
ANOTHER ONE OF THEIR SILLY SCENARIOS.  You glance about with a look of
irritation on your face.  WELL, I'LL SHOW THEM.  I'LL MAKE SHORT WORK OF THEIR
STUPID LITTLE PUZZLE...

THE ACORN COURT
An Interactive Text Adventure
Copyright (c) 1997 by Todd S. Murchison.
Release 3 / Serial number 970904 / Inform v6.13 Library 6/5
Standard interpreter 1.0

Court Yard
A good sized courtyard with an air of late British colonialism about it.  To the
west, in the direction the sun is setting, is a high, grey stone wall with an
ornate iron gate set into the rock.  The walls of a large stone mansion rise
several stories into the chilly evening air to the east, north, and south.  In
the east wall, two or three stories up, is a large window.  On top of the west
wall, above and to the right of the gate, is a large squirrels nest made of
sticks, twigs, and leaves.

A carpeting of old brown leaves from past winters rustle abou

In [25]:
textworld.play("./textplayer/games/acorncourt.z5", RandomAgent())

NameError: name 'HumanAgent' is not defined

In [34]:
import textworld.gym
env_id = textworld.gym.register_game("./textplayer/games/acorncourt.z5")

In [35]:
import gym
env = gym.make(env_id)

In [36]:
obs, infos = env.reset()
print(obs)


GREAT.  THEY'VE DONE IT TO ME AGAIN.  You think to yourself. THEY'VE STUCK ME IN ANOTHER ONE OF THEIR SILLY SCENARIOS.  You glance about with a look of irritation on your face.  WELL, I'LL SHOW THEM.  I'LL MAKE SHORT WORK OF THEIR STUPID LITTLE PUZZLE...

THE ACORN COURT
An Interactive Text Adventure
Copyright (c) 1997 by Todd S. Murchison.
Release 3 / Serial number 970904 / Inform v6.13 Library 6/5
Standard interpreter 1.0

Court Yard
A good sized courtyard with an air of late British colonialism about it.  To the west, in the direction the sun is setting, is a high, grey stone wall with an ornate iron gate set into the rock.  The walls of a large stone mansion rise several stories into the chilly evening air to the east, north, and south.  In the east wall, two or three stories up, is a large window.  On top of the west wall, above and to the right of the gate, is a large squirrels nest made of sticks, twigs, and leaves.

A carpeting of old brown leaves from past winters rustle abou

In [40]:
obs, score, done, infos = env.step("take well")
print(obs)

That's not a verb I recognise.


In [ ]:

import numpy as np

import textworld


class CustomAgent(textworld.Agent):
    def __init__(self, seed=1234):
        self.seed = seed
        self.rng = np.random.RandomState(self.seed)
        self.actions = ["north", "south", "east", "west", "up", "down",
                        "look", "inventory", "take all", "YES", "wait",
                        "take", "drop", "eat", "attack"]

    def reset(self, env):
        env.display_command_during_render = True

    def act(self, game_state, reward, done):
        action = self.rng.choice(self.actions)
        if action in ["take", "drop", "eat", "attack"]:
            words = game_state.feedback.split()  # Observed words.
            words = [w for w in words if len(w) > 3]  # Ignore most stop words.
            if len(words) > 0:
                action += " " + self.rng.choice(words)

        return action

In [41]:
import textworld
request_infos = textworld.EnvInfos(admissible_commands=True, entities=True)

# Requesting additional information should be done when registering the game.
env_id = textworld.gym.register_game("./textplayer/games/acorncourt.z5", request_infos)
env = gym.make(env_id)

obs, infos = env.reset()
print("Entities: {}".format(infos["entities"]))
print("Admissible commands:\n  {}".format("\n  ".join(infos["admissible_commands"])))

Entities: None


TypeError: can only join an iterable

In [33]:
play(RandomAgent(), "./textplayer/games/acorncourt.z5")

acorncourt.z5

ValueError: a must be 1-dimensional or an integer

In [ ]:
# We report the score and steps averaged over 10 playthroughs.
play(RandomAgent(), "./games/rewardsDense_goalDetailed.ulx")    # Dense rewards
play(RandomAgent(), "./games/rewardsBalanced_goalDetailed.ulx") # Balanced rewards
play(RandomAgent(), "./games/rewardsSparse_goalDetailed.ulx")   # Sparse rewards